<a href="https://colab.research.google.com/github/KevinTheRainmaker/MLOps/blob/main/MLOps_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GCP를 사용한 대용량 데이터셋 처리

In [ ]:
# cloud storage client library 설치
!pip install -q --upgrade google-cloud-storage

     |████████████████████████████████| 106 kB 4.5 MB/s 
     |████████████████████████████████| 75 kB 2.6 MB/s 
     |████████████████████████████████| 109 kB 52.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-translate 1.5.0 requires google-api-core[grpc]<2.0.0dev,>=1.6.0, but you have google-api-core 2.3.2 which is incompatible.
google-cloud-translate 1.5.0 requires google-cloud-core<2.0dev,>=1.0.0, but you have google-cloud-core 2.2.1 which is incompatible.
google-cloud-language 1.2.0 requires google-api-core[grpc]<2.0.0dev,>=1.6.0, but you have google-api-core 2.3.2 which is incompatible.
google-cloud-firestore 1.7.0 requires google-api-core[grpc]<2.0.0dev,>=1.14.0, but you have google-api-core 2.3.2 which is incompatible.
google-cloud-firestore 1.7.0 requires google-cloud-core<2.0dev,>=1.0.3, but you have google-cloud-core 2.2.1 which

In [ ]:
!pip install -q apache_beam

     |████████████████████████████████| 9.9 MB 4.4 MB/s 
     |████████████████████████████████| 62 kB 642 kB/s 
     |████████████████████████████████| 151 kB 37.0 MB/s 
     |████████████████████████████████| 2.3 MB 32.3 MB/s 
     |████████████████████████████████| 45 kB 2.4 MB/s 
     |████████████████████████████████| 247 kB 48.3 MB/s 
     |████████████████████████████████| 1.1 MB 32.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.1.1 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
google-cloud-bigquery 1.21.0 requires google-cloud-core<2.0dev,>=1.0.3, but you have google-cloud-core 2.2.1 which is incompatible.
google-cloud-bigquery 1.21.0 requires google-resumable-media!=0.4.0,<0.5.0dev,>=0.3.1, but you have 

In [ ]:
!pip install -q tensorflow-data-validation

     |████████████████████████████████| 1.4 MB 4.2 MB/s 
     |████████████████████████████████| 19.1 MB 60.2 MB/s 
     |████████████████████████████████| 294 kB 42.9 MB/s 
     |████████████████████████████████| 48 kB 4.0 MB/s 
     |████████████████████████████████| 171 kB 53.1 MB/s 
     |████████████████████████████████| 255 kB 51.6 MB/s 
     |████████████████████████████████| 173 kB 46.9 MB/s 
     |████████████████████████████████| 110 kB 34.0 MB/s 
     |████████████████████████████████| 267 kB 48.9 MB/s 
     |████████████████████████████████| 435 kB 41.6 MB/s 
     |████████████████████████████████| 83 kB 1.7 MB/s 
     |████████████████████████████████| 144 kB 34.1 MB/s 
     |████████████████████████████████| 180 kB 47.0 MB/s 
     |████████████████████████████████| 183 kB 36.8 MB/s 
     |████████████████████████████████| 517 kB 36.1 MB/s 
     |████████████████████████████████| 267 kB 34.8 MB/s 
     |████████████████████████████████| 267 kB 10.1 MB/s 
     |████████████

In [ ]:
!pip install -q tfx==1.2.0

     |████████████████████████████████| 2.4 MB 4.4 MB/s 
     |████████████████████████████████| 1.7 MB 37.2 MB/s 
     |████████████████████████████████| 406 kB 38.3 MB/s 
     |████████████████████████████████| 1.2 MB 27.0 MB/s 
     |████████████████████████████████| 1.4 MB 7.4 MB/s 
     |████████████████████████████████| 147 kB 47.8 MB/s 
     |████████████████████████████████| 6.5 MB 34.8 MB/s 
     |████████████████████████████████| 454.4 MB 22 kB/s 
     |████████████████████████████████| 19.0 MB 1.3 MB/s 
     |████████████████████████████████| 17.7 MB 315 kB/s 
     |████████████████████████████████| 1.7 MB 42.5 MB/s 
     |████████████████████████████████| 1.3 MB 32.0 MB/s 
     |████████████████████████████████| 40 kB 4.5 MB/s 
     |████████████████████████████████| 189 kB 54.2 MB/s 
     |████████████████████████████████| 54 kB 2.3 MB/s 
     |████████████████████████████████| 49 kB 4.4 MB/s 
     |████████████████████████████████| 636 kB 45.3 MB/s 
     |████████████████

In [ ]:
def implicit():
    from google.cloud import storage

    # 클라이언트 생성 시 인증정보를 명시하지 않았을 경우
    # 클라이언트 라이브러리는 환경변수에서 인증정보를 탐색한다
    storage_client = storage.Client()

    # authentificated API request 생성
    buckets = list(storage_client.list_buckets())
    print(buckets)

In [ ]:
from apache_beam.options.pipeline_options import PipelineOptions, GoogleCloudOptions, StandardOptions

options = PipelineOptions()
google_cloud_options = options.view_as(GoogleCloudOptions)

# 프로젝트 ID 설정
google_cloud_options.project = 'YOUR_GCP_PROJECT_ID'

# 작업 이름 지정
google_cloud_options.job_name = 'YOUR_JOB_NAME'

# 스테이징 및 임시 파일의 저장소 버킷 지정
google_cloud_options.staging_location = 'gs://YOUR_GCP_BUCKET/staging'
google_cloud_options.temp_location = 'gs://YOUR_GCP_BUCKET/tmp'

options.view_as(StandardOptions).runner = 'DataflowRunner'

In [ ]:
from apache_beam.options.pipeline_options import SetupOptions

setup_options = options.view_as(SetupOptions)
setup_options.extra_packages = [
                                'PATH_TO_TFDV',
                                '-0.22.0-cp37-cp37m-manylinux2010_x86_64.whl']

In [ ]:
import tensorflow_data_validation as tfdv

dataset_path = 'gs://YOUR_GCP_BUCKET/train_reviews.tfrecord'
output_path = 'gs://YOUR_GCP_BUCKET/'
tfdv.generate_statistics_from_tfrecord(dataset_path,
                                       output_path=output_path,
                                       pipeline_options=options)

In [ ]:
from tfx.components import StatisticsGen
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

context = InteractiveContext()

statistics_gen = StatisticsGen(examples=example_gen.outputs['example'])
context.run(statistics_gen)

In [ ]:
context.show(statistics_gen.outputs['statistics'])

In [ ]:
from tfx.components import SchemaGen

schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'],
                       infer_feature_shape=True)
context.run(schema_gen)

In [ ]:
from tfx.components import ExampleValidator

example_validator = ExampleValidator(statistics=statistics_gen.outputs['statistics'],
                                     schema=schema_gen.outputs['schema'])
context.run(example_validator)